In [ ]:
#Mounting google drive

from google.colab import drive
drive.mount('/content/gdrive')

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144542 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.18-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [ ]:
!mkdir -p drive
!ls

adc.json  drive  sample_data


In [ ]:
import numpy as np
from skimage import io
import tensorflow as tf
from tensorflow.keras import models
import matplotlib.pyplot as plt
import time
%matplotlib inline
tf.compat.v1.enable_v2_behavior()
tf.keras.backend.set_floatx('float64')

def open_image ():
    content = io.imread('drive/Style_Transfer_Data/content.jpg')[:,:,:3]
    content = np.expand_dims(content, axis= 0)
    #content = tf.image.convert_image_dtype(content, dtype=tf.float64, saturate=False)
    
    style = io.imread('drive/Style_Transfer_Data/style.jpg')[:,:,:3]
    style = np.expand_dims(style, axis= 0)
    #style = tf.image.convert_image_dtype(style, dtype=tf.float64, saturate=False)
    
    return (content, style)

def convert_to_real_image(cimg):
    img = cimg[:,:,:,:]
    img = np.squeeze(img, 0)
    img[:, :, 0] = img[:, :, 0] + 103.939
    img[:, :, 1] = img[:, :, 1] + 116.779
    img[:, :, 2] = img[:, :, 2] + 123.68
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype('uint8')
    return img

def content_loss(content, target):
    return tf.reduce_mean(tf.square(content - target))

def gram_matrix(input_tensor):
    channels = int(input_tensor.shape[-1])
    a = tf.reshape(input_tensor, [-1, channels])
    n = tf.shape(a)[0]
    gram = tf.matmul(a, a, transpose_a=True)
    return gram / tf.cast(n, tf.float64)

def compute_loss(content_copy, content_img_content_features, style_img_gram_features):
    style_score = 0
    content_score = 0


    model_outputs = model(content_copy)
    content_copy_gram_features = [gram_matrix(style_layer[0]) for style_layer in model_outputs[:num_style_layers]]
    content_copy_content_features = [content_layer[0] for content_layer in model_outputs[num_style_layers:]]

    weight_per_style_layer = 1.0 / float(num_style_layers)
    for content_copy_gram_layer, style_img_gram_layer in zip(content_copy_gram_features, style_img_gram_features):
        style_score += weight_per_style_layer * content_loss(content_copy_gram_layer, style_img_gram_layer)

    weight_per_content_layer = 1.0 / float(num_content_layers)
    for content_copy_content_layer, content_img_content_layer in zip(content_copy_content_features, content_img_content_features):
        content_score += weight_per_content_layer * content_loss(content_copy_content_layer, content_img_content_layer)


    loss = (style_score * style_weight) + (content_score * content_weight)
    return loss

def compute_grads(content_copy, style_img_content_features, style_img_gram_features):
    with tf.GradientTape() as tape:
        all_loss = compute_loss(content_copy, style_img_content_features, style_img_gram_features)
    total_loss = all_loss
    return tape.gradient(total_loss, content_copy), all_loss

def get_model():
    vgg = tf.keras.applications.vgg19.VGG19(include_top = False, weights='imagenet')
    vgg.trainable = False
    style_output = [vgg.get_layer(name).output for name in style_layers]
    content_output = [vgg.get_layer(name).output for name in content_layers]
    model_output = style_output + content_output
    return models.Model(vgg.input, model_output)

if __name__=='__main__':
    
    content, style = open_image()
    
    content = tf.keras.applications.vgg19.preprocess_input(content)
    style = tf.keras.applications.vgg19.preprocess_input(style)
    
    content_copy = content.copy()
    content_copy = tf.Variable(content_copy)
    
    num_iterations = 1001
    content_weight = 1e3
    style_weight = 1e-2
    content_layers = ['block5_conv2']
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
    
    num_content_layers = len(content_layers)
    num_style_layers = len(style_layers)
    
    model = get_model()
    for layer in model.layers:
        layer.trainable = False

    norm_means = np.array([103.939, 116.779, 123.68])
    min_vals = - norm_means
    max_vals = 255 - norm_means


    style_img_output = model(style)
    content_img_output = model(content)
    
    style_img_gram_features = [gram_matrix(style_layer[0]) for style_layer in style_img_output[:num_style_layers]]
    content_img_content_features = [content_layer[0] for content_layer in content_img_output[num_style_layers:]]
    
    opt = tf.optimizers.Adam(learning_rate=5, beta_1=0.99, epsilon=1e-1)
    
    display_num = 100
    # For displaying intermediate images 
    iter_count = 1

    # For displaying
    plt.figure(figsize=(15, 15))
    num_rows = (num_iterations / display_num) // 5
    start_time = time.time()
    global_start = time.time()

    
    for i in range(num_iterations):
        print ('iter' + str(i) +'\n')
        grads, all_loss = compute_grads(content_copy, content_img_content_features, style_img_gram_features)
        loss = all_loss
        opt.apply_gradients([(grads, content_copy)])
        clipped = tf.clip_by_value(content_copy, min_vals, max_vals)
        content_copy.assign(clipped)
        end_time = time.time() 
        plot_img = content_copy.numpy()
        plot_im = convert_to_real_image(plot_img)
        
        if i % 100 == 0:
            io.imsave('drive/Style_Transfer_Data/NST_' + str(i) + '.jpg', plot_im)
            print("Loss ", loss, " At Iteration # ", i)
        # Display intermediate images
        
        if iter_count > num_rows * 5: 
            
            plt.subplot(num_rows, 5, iter_count)
            # Use the .numpy() method to get the concrete numpy array
            plt.imshow(plot_im)
            plt.title('Iteration {}'.format(i + 1))

            iter_count += 1
            

iter0

Loss  tf.Tensor(1777617757.7609696, shape=(), dtype=float64)  At Iteration #  0
iter1

iter2

iter3

iter4

iter5

iter6

iter7

iter8

iter9

iter10

iter11

iter12

iter13

iter14

iter15

iter16

iter17

iter18

iter19

iter20

iter21

iter22

iter23

iter24

iter25

iter26

iter27

iter28

iter29

iter30

iter31

iter32

iter33

iter34

iter35

iter36

iter37

iter38

iter39

iter40

iter41

iter42

iter43

iter44

iter45

iter46

iter47

iter48

iter49

iter50

iter51

iter52

iter53

iter54

iter55

iter56

iter57

iter58

iter59

iter60

iter61

iter62

iter63

iter64

iter65

iter66

iter67

iter68

iter69

iter70

iter71

iter72

iter73

iter74

iter75

iter76

iter77

iter78

iter79

iter80

iter81

iter82

iter83

iter84

iter85

iter86

iter87

iter88

iter89

iter90

iter91

iter92

iter93

iter94

iter95

iter96

iter97

iter98

iter99

iter100

Loss  tf.Tensor(28117312.286231432, shape=(), dtype=float64)  At Iteration #  100
iter101

iter102

iter103

iter104

ite

<Figure size 1080x1080 with 0 Axes>